In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
user='root'
password='root'
host='localhost'
port=5432
db='btpn'

In [51]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [52]:
engine.connect()

In [53]:
df = pd.read_csv('data/customer_data_history.csv')
df

,CLIENTNUM,idstatus,Customer_Age,Gender,Dependent_count,Educationid,Maritalid,Income_Category,card_categoryid,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio
0,768805383,1,45,M,3,1,1,$60K - $80K,1,39,5,1,3,12691.0,777,11914.0,1144,42,0.061
1,818770008,1,49,F,5,2,2,Less than $40K,1,44,6,1,2,8256.0,864,7392.0,1291,33,0.105
2,713982108,1,51,M,3,2,1,$80K - $120K,1,36,4,1,0,3418.0,0,3418.0,1887,20,0.000
3,769911858,1,40,F,4,1,3,Less than $40K,1,34,3,4,1,3313.0,2517,796.0,1171,20,0.760
4,709106358,1,40,M,3,3,1,$60K - $80K,1,21,5,1,0,4716.0,0,4716.0,816,28,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,1,50,M,2,2,2,$40K - $60K,1,40,3,2,3,4003.0,1851,2152.0,15476,117,0.462
10123,710638233,2,41,M,2,4,0,$40K - $60K,1,25,4,2,3,4277.0,2186,2091.0,8764,69,0.511
10124,716506083,2,44,F,1,1,1,Less than $40K,1,36,5,3,4,5409.0,0,5409.0,10291,60,0.000
10125,717406983,2,30,M,2,2,3,$40K - $60K,1,36,4,3,3,5281.0,0,5281.0,8395,62,0.000


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   idstatus                  10127 non-null  int64  
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  object 
 4   Dependent_count           10127 non-null  int64  
 5   Educationid               10127 non-null  int64  
 6   Maritalid                 10127 non-null  int64  
 7   Income_Category           10127 non-null  object 
 8   card_categoryid           10127 non-null  int64  
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit              10127 non-null  float64
 14  Total_

show the table schemas

In [56]:
print(pd.io.sql.get_schema(df, name='customer_history', con=engine))


CREATE TABLE customer_history (
	"CLIENTNUM" BIGINT, 
	idstatus BIGINT, 
	"Customer_Age" BIGINT, 
	"Gender" TEXT, 
	"Dependent_count" BIGINT, 
	"Educationid" BIGINT, 
	"Maritalid" BIGINT, 
	"Income_Category" TEXT, 
	card_categoryid BIGINT, 
	"Months_on_book" BIGINT, 
	"Total_Relationship_Count" BIGINT, 
	"Months_Inactive_12_mon" BIGINT, 
	"Contacts_Count_12_mon" BIGINT, 
	"Credit_Limit" FLOAT(53), 
	"Total_Revolving_Bal" BIGINT, 
	"Avg_Open_To_Buy" FLOAT(53), 
	"Total_Trans_Amt" BIGINT, 
	"Total_Trans_Ct" BIGINT, 
	"Avg_Utilization_Ratio" FLOAT(53)
)




make the columns

In [59]:
df.head(0).to_sql(name='customer_history', con=engine, if_exists='replace')

0

append the data

In [60]:
df.to_sql(name='customer_history', con=engine, if_exists='append')

127

total rows

In [61]:
len(df)

10127

insert dim table

In [74]:
dim_table = ['fact_customer_data_history.csv', 'category_db.csv', 'education_db.csv', 'marital_db.csv', 'status_db.csv']

In [75]:
for file_name in dim_table:
    path = f"data/{file_name}"
    dim_name = file_name.split('_')[0]
    
#     print(f"""
#     {path}
#     {dim_name}
#     """)
    df = pd.read_csv(path)
    
    df.head(0).to_sql(name=dim_name, con=engine, if_exists='replace')
    df.to_sql(name=dim_name, con=engine, if_exists='append')
    
    print(f"{dim_name} inserted!")

fact inserted!
category inserted!
education inserted!
marital inserted!
status inserted!


In [76]:
schema = pd.read_csv('data/schema.csv')
schema

,Unnamed: 0,Clientnum,Idstatus,Customer_age,Gender,Dependent_count,Educationid,Maritalid,Income_category,Card_categoryid,Month_on_book,Relationship_in_count,Months_inactive_in_12_month,Contacts_Count_12_mon,Credit_limit,Total_Revolving_Bal,Avg_open_to_buy,Total_trans_amt,Total_trans_ct,Avg_utilization_ratio
0,0,nomor id client,keterangan status customer,umur customer,jenis kelamin customer,yang menjadi tanggungan customer,keterangan jenjang Pendidikan customer,status pernikahan customer,ketgori penghasilan customer,jenis kartu kredit customer,periode berhubungan dengan bank,total product yang dipegang customer,jumlah bulan tidak aktif selama 12 bulan terakhir,total dihubungi bank dalam 12 bulan terakhir,limit credit,total saldo bergulir pada kartu kredit,membeli dengan kartu kredit dalam 12 bulan ter...,jumlah transaksi,frekuensi transaksi,rata rata rasio penggunaan kartu kredit
